### Imports

In [17]:
import requests
import json
from bs4 import BeautifulSoup

2. Sending HTTP request to acces first page with options

In [18]:
url = "https://www.ceneo.pl/84514582#tab=reviews_scroll"

response = requests.request(method="GET" , url=url)
response.status_code

200

3. Checking the code of Http responce

In [19]:
response.status_code

200

4. Parse the HTML code of first page with options

In [20]:
page_dom= BeautifulSoup(response.text, "html.parser")
print(page_dom.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7  non-direct ab-inactive" lang="pl"> <![endif]-->
<!--[if IE 7]> <html class="no-js lt-ie9 lt-ie8  non-direct ab-inactive" lang="pl"> <![endif]-->
<!--[if IE 8]> <html class="no-js lt-ie9  non-direct ab-inactive" lang="pl"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js non-direct ab-inactive" lang="pl">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <link href="https://image.ceneo.pl" rel="preconnect"/>
  <link href="https://image.ceneostatic.pl" rel="preconnect"/>
  <link href="https://www.google-analytics.com" rel="preconnect"/>
  <link href="https://www.googleadservices.com" rel="preconnect"/>
  <link href="https://adservice.google.com" rel="preconnect"/>
  <link href="https://www.google.pl" rel="preconnect"/>
  <link href="https://www.google.com" rel="preconnect"/>
  <link href="https://cdn.ampproject.org" rel="preconnect"/>
  <title>
   

5. Extract required data from parsed code

In [26]:
opinion = page_dom.select_one("div.js_product-review")
print(type(page_dom))
print(type(opinion))
print(opinion.prettify())

<class 'bs4.BeautifulSoup'>
<class 'bs4.element.Tag'>
<div class="user-post user-post__card user-post--highlight js_product-review" data-entry-id="19042377">
 <header class="user-post__header">
  <div class="js_lazy user-post__avatar user-rank--ceneoai user-rank__avatar" data-bg="https://image.ceneostatic.pl/data/avatars/27/93/20e6-0e21-407d-a166-45492bb22049_avatar.jpg">
  </div>
 </header>
 <div class="user-post__body">
  <div class="user-post__aisummary-author-name">
   CeneoAI
  </div>
  <div class="user-post__info">
   <div class="d-flex align-items-center justify-content-end">
    <div class="js_product-review-usefulness vote">
     <button class="vote-yes js_product-review-vote js_vote-yes" data-new-icon="vote-up" data-product-id="84514582" data-review-id="19042377" data-total-vote="0" data-url="SetOpinionVote" data-vote="1" data-voted="false">
      <span id="votes-yes-19042377">
       0
      </span>
     </button>
     <button class="vote-no js_product-review-vote js_vote-no